# 01 – Data Quality Assessment

This notebook analyses the raw credit application dataset for data quality issues across the following dimensions:

- Completeness
- Consistency
- Validity
- Accuracy

All issues are quantified (counts and percentages) and mapped to governance implications.

## 1. Data Loading & Structural Inspection
- Load raw JSON
- Inspect nested structure
- Examine column names and data types

## 2. Completeness Analysis
- Missing values per column
- Incomplete nested objects
- % affected records

## 3. Consistency & Type Validation
- Data type mismatches
- Inconsistent categorical encoding (e.g., gender formats)
- Date format inconsistencies

## 4. Validity Checks
- Impossible values (e.g., negative income, negative credit history)
- Logical inconsistencies (e.g., interest rate assigned when rejected)

## 5. Duplicate Record Detection

## 6. Remediation Strategy
- Cleaning logic
- Standardisation decisions
- Governance implications

---

## Setup & Connection

In [30]:
# Import required libraries
!pip install pymongo 

In [31]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

# Establish connection to Local MongoDB
client = MongoClient('mongodb://localhost:27017/')

# Select database and collection
db = client['novacred']
collection = db['credit_applications']

# Fetch a sample document to verify connection
print(f"Connected to database: {db.name}")
print(f"Collection: {collection.name}")
print(f"Document count: {collection.count_documents({})}")

Connected to database: novacred
Collection: credit_applications
Document count: 500


**Ingestion Audit Note:**
 
MongoDB automatically rejected 2 records (app_042, app_001) during import due to E11000 duplicate key errors on the _id field. Baseline record count established at 500 documents.

In [36]:
import json
from pathlib import Path
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

# Establish Connection to Local MongoDB & select database and collection
client = MongoClient('mongodb://localhost:27017/')
db = client['novacred']
collection = db['credit_applications']

# Reset Collection to ensure a clean slate
collection.drop()
print("Previous collection dropped.")

# Load JSON Data using canonical path based on current working directory
cwd = Path.cwd()
if cwd.name == 'notebooks':
    data_path = cwd.parent / 'data' / 'raw_credit_applications.json'
else:
    data_path = cwd / 'data' / 'raw_credit_applications.json'

if not data_path.exists():
    raise FileNotFoundError(f"Could not locate file: {data_path}")

with data_path.open('r') as file:
    raw_data = json.load(file)

# Execute Fault-Tolerant Ingestion
successful_inserts = 0
rescued_duplicates = 0

for doc in raw_data:
    try:
        collection.insert_one(doc)
        successful_inserts += 1
    except DuplicateKeyError:
        # Bypassing the constraint by altering the primary key
        doc['_id'] = f"{doc['_id']}_duplicate"
        collection.insert_one(doc)
        rescued_duplicates += 1

print("\n--- Ingestion Audit ---")
print(f"Standard Records Inserted: {successful_inserts}")
print(f"Duplicate Records Rescued: {rescued_duplicates}")
print(f"Total Database Document Count: {collection.count_documents({})}")

Previous collection dropped.

--- Ingestion Audit ---
Standard Records Inserted: 500
Duplicate Records Rescued: 2
Total Database Document Count: 502


---

## Quick Data Overview

In [ ]:
# View a sample document from the collection
sample = collection.find_one()
pprint(sample)

{'_id': 'app_024',
 'applicant_info': {'date_of_birth': '1983-04-25',
                    'email': 'thomas.lee6@protonmail.com',
                    'full_name': 'Thomas Lee',
                    'gender': 'Male',
                    'ip_address': '192.168.175.67',
                    'ssn': '194-35-1833',
                    'zip_code': '10077'},
 'decision': {'approved_amount': 34000,
              'interest_rate': 4.3,
              'loan_approved': True},
 'financials': {'annual_income': 103000,
                'credit_history_months': 70,
                'debt_to_income': 0.35,
                'savings_balance': 0},
 'spending_behavior': [{'amount': 575, 'category': 'Fitness'}]}


---